## Tabnet

In [0]:
def upgrade_runtime_ram():
    meminfo = subprocess.getoutput('cat /proc/meminfo').split('\n')

    memory_info = {entry.split(':')[0]: int(entry.split(':')[1].replace(' kB','').strip()) for entry in meminfo}

    if memory_info['MemTotal'] > 17000000:
        return

    a = []
    while(1):
        a.append('1')

In [0]:
def restart_runtime():
    os.kill(os.getpid(), 9)

In [0]:
def setup_rapids():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    device_name = pynvml.nvmlDeviceGetName(handle)
    if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
        print("Wrong GPU - Restarting Runtime")
        restart_runtime()


    # clone RAPIDS AI rapidsai-csp-utils scripts repo
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git

    # install RAPIDS
    !bash rapidsai-csp-utils/colab/rapids-colab.sh 0.13


    # set necessary environment variables 
    dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
    sys.path

    # update pyarrow & modules 
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [0]:
def setup_conda():
    if not 'Miniconda3-4.5.4-Linux-x86_64.sh' in os.listdir():
        !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

    if not ('EPFL-Capstone-Project' in os.listdir()) and (os.getcwd().split('/')[-1] != 'EPFL-Capstone-Project'):
        !git clone https://github.com/helmigsimon/EPFL-Capstone-Project  
    if 'EPFL-Capstone-Project' in os.listdir():
        os.chdir('EPFL-Capstone-Project')

    !conda env create -f environment.yml
    !conda activate exts-ml

In [0]:
def setup_drive():
    #Mounting Google Drive
    global drive
    from google.colab import drive
    drive.mount('/content/drive')

In [7]:
try:
    import sys,os,subprocess
    
    upgrade_runtime_ram()
    setup_drive()

    #Setting up PyPi Packages
    !pip install sparse-dot-topn pdpipe category-encoders tabnet[gpu]
    import sparse_dot_topn.sparse_dot_topn as ct
    import pdpipe as pdp
    import category_encoders
    import tabnet

    #Setting up Conda Packages
    setup_conda()
    !conda install -y tensorflow==2.1

    
    #Initializing NLTK
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    
except ModuleNotFoundError as e:
    print(e)
    print('Not in colab environment, continuing to run locally')
    from umap import UMAP

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 102kB 6.5MB/s 
     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 450kB 42.4MB/s 
     |████████████████████████████████| 3.9MB 43.2MB/s 
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.2.9-cp36-cp36m-linux_x86_64.whl size=322963 sha256=8ebf83df4f36389f1

--2020-04-23 17:45:12--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2020-04-23 17:45:12--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   170MB/s    in 0.3s    

2020-04-23 17:45:12 (170 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_

In [0]:
import tensorflow as tf
import tabnet
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders import LeaveOneOutEncoder
from sklearn.compose import ColumnTransformer

In [9]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [0]:
from lib.processing import load_from_pkl
from data.util.paths import DATA_PATH
from lib.pipelines import *

In [0]:
api_df, extracted_df = load_from_pkl('api',DATA_PATH), load_from_pkl('extracted',DATA_PATH)

In [0]:
api_df = api_pipe.fit_transform(api_df)

In [13]:
extracted_df = extracted_pipe.fit_transform(extracted_df)

In [0]:
with np.load(os.path.join(DATA_PATH,'high_level_features_labelled.npz')) as data:
    image_embedding_df = pd.concat([pd.DataFrame(data[section]) for section in ('release_id','bitmap','features')],axis=1)
    image_embedding_df.columns = ['release_id', 'bitmap'] + ['feature_%s' % i for i in range(1,1281)]

In [0]:
scaler = StandardScaler()
pca = PCA(n_components=10)
image_embeddings_scaled = scaler.fit_transform(image_embedding_df.loc[:,['feature_%s' % i for i in range(1,1281)]])

In [0]:
image_embeddings_reduced = pca.fit_transform(image_embeddings_scaled)

In [0]:
image_embeddings_reduced = pd.concat([
      image_embedding_df.loc[:,'release_id'],
      pd.DataFrame(
          image_embeddings_reduced,
          columns = ['images_umap_%s' % i for i in range(image_embeddings_reduced.shape[1])]
      )],
      axis=1
)


Combining datasets

In [0]:
df = api_df.merge(extracted_df,how='inner',on='release_id')
df = df.merge(image_embeddings_reduced,how='inner',on='release_id')

In [0]:
del api_df, extracted_df, image_embedding_df, image_embeddings_scaled, image_embeddings_reduced

In [0]:
record_store_tabnet_transformer = ColumnTransformer(transformers=[
    ('year_encoder', OneHotEncoder(dtype=np.uint8,handle_unknown='ignore'), ['year']),
    ('format_name', OneHotEncoder(dtype=np.uint8,handle_unknown='ignore'), ['format_name'])
], remainder='passthrough')

In [0]:
record_store_tabnet_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title',
]

In [0]:
record_store_tabnet_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','master_id'])),
    ('record_store_column_remover', ColumnRemover(record_store_tabnet_removal_columns)),
    ('preprocessing',record_store_tabnet_transformer),
    ('scaler', StandardScaler()),
])

In [0]:
df_arr = record_store_tabnet_pipe.fit_transform(df.drop('market_value',axis=1),df.market_value)

In [0]:
df_pretensor = np.concatenate([df_arr,np.log(df['market_value']).values.reshape(-1,1)],axis=1)
np.random.shuffle(df_pretensor)

In [0]:
ds = tf.data.Dataset.from_tensor_slices(df_pretensor)

In [0]:
def transform(ds):
    unstacked = tf.unstack(ds)
    features = unstacked[:-1]
    
    x = dict(zip(col_names,features))
    y = unstacked[-1]
    
    return x,y

In [0]:
col_names = ['column_%s' % i for i in range(975)]

In [0]:
train_size = int(0.8*len(df))
BATCH_SIZE = 1000
ds_train = ds.take(train_size)
ds_train = ds_train.map(transform)
ds_train = ds_train.batch(BATCH_SIZE)

In [0]:
ds_test = ds.skip(train_size)
ds_test = ds_test.map(transform)
ds_test = ds_test.batch(BATCH_SIZE)

In [0]:
feature_columns = [tf.feature_column.numeric_column(column) for column in col_names]

In [0]:
model = tabnet.TabNetRegressor(
    feature_columns,
    feature_dim=64,
    output_dim=64,
    num_regressors=len(feature_columns),
    num_decision_steps=2,
    relaxation_factor=1,
    sparsity_coefficient=1e-2,    
)

In [0]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01,decay_steps=100,decay_rate=0.9,staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)

In [0]:
model.compile(optimizer,loss='mean_squared_error',metrics=['mean_absolute_error'])

In [0]:
model.fit(ds_train,epochs=50,validation_data=ds_test,verbose=100)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

